### Preliminary Data Wrangling. The objectives of this file are to connect REF files with Raw Data Downloads

Importing necessary packages 

In [5]:
#Importing necessary packages
import pandas as pd
import os 
import numpy as np
import warnings
import ace_tools as tools
warnings.simplefilter("ignore")

# Setting working directory and reading in files: 
os.chdir(r"C:\Users\mmath\OneDrive\Desktop\Capstone\raw_data\raw_data2023")


ModuleNotFoundError: No module named 'ace_tools'

Loading in all raw data & REF files and the REF files by storing them into dictionaries  

In [54]:
### Storing all raw data files in a dictionary ### 
# Get a list of all CSV files in the directory (These are the raw data files )
csv_files = [f for f in os.listdir() if f.endswith('.csv')]

# Read each CSV file and store it in a dictionary with lowercase keys
RAWDATA_23 = {file.split('.')[0].lower(): pd.read_csv(file) for file in csv_files}


### Storing all ref files in a dictionary ### 
# Define file path for the Excel file
file_path = r"TAPR_district_adv_2023.xlsx"

# Get all sheet names
sheetname_ref = pd.ExcelFile(file_path).sheet_names

# Read all sheets into a dictionary with modified keys
REF_23 = {sheet + "_ref2023": pd.read_excel(file_path, sheet_name=sheet) for sheet in sheetname_ref}

# Display the updated dictionary keys
print("Updated sheet names:", REF_23.keys())

Updated sheet names: dict_keys(['dref_ref2023', 'diststaar1_ref2023', 'diststaar2_ref2023', 'diststaar3_ref2023', 'diststaar4_ref2023', 'diststaar5_ref2023', 'diststaar6_ref2023', 'diststaarv_ref2023', 'distpart1_ref2023', 'distpart1a_ref2023', 'distpart2_ref2023', 'diststaar_add1_ref2023', 'distgrad_ref2023', 'distcomp_ref2023', 'distperf1_ref2023', 'distperf2_ref2023', 'distperf3_ref2023', 'distprof_ref2023'])


Comparing the shapes between the REF and raw_data files to see if the ref has all column names 

In [56]:
# Create a dictionary to store comparison results
shape_comparison = {}

# Iterate through each file in RAWDATA_23
for raw_key, raw_df in RAWDATA_23.items():
    # Extract the base name before the first underscore (_)
    base_name = raw_key.split("_")[0]
    
    # Find the matching key in REF_23 (case-insensitive match)
    matching_key = next((key for key in REF_23 if key.lower().startswith(base_name.lower())), None)
    
    if matching_key:
        # Get the corresponding DataFrame from REF_23
        ref_df = REF_23[matching_key]
        
        # Compare shapes
        shape_comparison[raw_key] = {
            "RAWDATA_23_shape": raw_df.shape,
            "REF_23_shape": ref_df.shape,
            "Match": raw_df.shape == ref_df.shape
        }
    else:
        shape_comparison[raw_key] = {
            "RAWDATA_23_shape": raw_df.shape,
            "REF_23_shape": None,
            "Match": False
        }

# Convert results into a DataFrame for easy viewing
comparison_df = pd.DataFrame.from_dict(shape_comparison, orient='index')

# Display the comparison results
tools.display_dataframe_to_user(name="Shape Comparison Results", dataframe=comparison_df)


ModuleNotFoundError: No module named 'ace_tools'